In [1]:
library(Seurat)
library(cowplot)
library(scran)

Loading required package: ggplot2
Loading required package: cowplot

Attaching package: ‘cowplot’

The following object is masked from ‘package:ggplot2’:

    ggsave

Loading required package: Matrix
Loading required package: BiocParallel
Loading required package: SingleCellExperiment
Loading required package: SummarizedExperiment
Loading required package: GenomicRanges
Loading required package: stats4
Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following objects are masked from ‘package:Matrix’:

    colMeans, colSums, rowMeans, rowSums, which

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

The following objects are masked from ‘package:base’:

  

In [52]:
data1_path = './pairwise/t4k.csv'
data2_path = './pairwise/pbmc4k.csv'

In [3]:
setwd("/Users/dariapodorskaja/Bachelor_thesis")


In [53]:
data1 <- read.table('./pairwise/t4k.csv')
data2 <- read.table('./pairwise/pbmc4k.csv')

In [23]:
raw.all <- cbind(data1, data2)

In [24]:
first.batch <- rep(c(TRUE, FALSE), c(ncol(data1), ncol(data2)))

In [67]:
# Making a plotting function.
plotFUN <- function(fname, Y, subset=NULL, ..., xlab="tSNE 1",ylab="tSNE 2",main="") {
  if (is.null(subset)) {
    subset <- seq_len(nrow(Y))
  }
  png(fname,width=900,height=700)
  par(mfrow=c(1,1),mar=c(6,6,4,2),cex.axis=2,cex.main=3,cex.lab=2.5)
  plot(Y[,1], Y[,2], cex=2,
       pch=ifelse(first.batch, 21, 1)[subset], 
       col=ifelse(first.batch, "black", "red")[subset],
       bg="white", xlab=xlab, ylab=ylab, main=main) 
    dev.off()
}

In [68]:
batchcolor=c("black","yellow")
plotFUNb <- function(fname, Y, subset=NULL, ...) {
  if (is.null(subset)) {
    subset <- seq_len(nrow(Y))
  }
  png(fname,width=900,height=700)
  par(mfrow=c(1,1),mar=c(6,6,4,2),cex.axis=2,cex.main=3,cex.lab=2.5)
  plot(Y[,1], Y[,2], cex=2,
       pch=ifelse(first.batch, 21, 1)[subset], 
       col=ifelse(first.batch, "black", "yellow")[subset],
       bg="white")#,  xlab="tSNE 1",ylab="tSNE 2")
  dev.off()
    }

In [27]:
X.unc <- raw.all

In [28]:
t.unc <- t(X.unc)


In [29]:
require(Rtsne)


In [30]:
set.seed(0)

In [31]:
all.dists.unc <- as.matrix(dist(t.unc))

Warning message in dist(t.unc):
“NAs introduced by coercion”

In [46]:
tsne.unc <- Rtsne(all.dists.unc, is_distance=TRUE, perplexity = 0.3)

Perplexity should be lower than K!


In [69]:
plotFUN("results/uncFA.png", tsne.unc$Y, main="Uncorrected",  xlab="tSNE 1",ylab="tSNE 2")

pdf 
  2

In [70]:
plotFUNb("results/uncFAb.png", tsne.unc$Y, main="Uncorrected",  xlab="tSNE 1",ylab="tSNE 2")

pdf 
  2

# Corrected plot

In [54]:
data <- lapply(c(data1_path,data2_path),function(fname){
    X = read.csv(fname,header=F,as.is=T)
})

In [55]:
mnn.out <- mnnCorrect(as.matrix(data[[1]]), as.matrix(data[[2]]), 
                          k=20, sigma=0.1,cos.norm.in=TRUE, cos.norm.out=TRUE, var.adj=TRUE,compute.angle=TRUE) # corrected values

In [71]:
# plot histogram of angles between batch vectors and 2 svds of the reference batch.
png(file="results/angles.png",width=900,height=700)
par(mfrow=c(1,1),mar=c(6,6,4,2),cex.axis=2,cex.main=3,cex.lab=2.5)
hist(mnn.out$angles[[2]],xlab="Angle",ylab="Frequency",main="")
dev.off()

pdf 
  2

In [72]:
# Generating a t-SNE plot.
set.seed(0)
all.dists.mnn <- as.matrix(dist(t.mnn))
tsne.mnn <- Rtsne(all.dists.mnn, is_distance=TRUE, perplexity = 90)
plotFUN("results/mnnFA.png", tsne.mnn$Y, main="MNN",  xlab="tSNE 1",ylab="tSNE 2")
plotFUNb("results/mnnFAb3.png", tsne.mnn$Y, main="MNN",  xlab="tSNE 1",ylab="tSNE 2")

pdf 
  2

pdf 
  2

In [73]:
set.seed(0)
tsne.mnn2 <- Rtsne(t.mnn, perplexity = 90, check_duplicates = FALSE)
plotFUN("results/mnnFA_conventsne.png", tsne.mnn2$Y, main="MNN",  xlab="tSNE 1",ylab="tSNE 2")

pdf 
  2